<a href="https://colab.research.google.com/github/BPMvrn/Work_OM_public/blob/main/%D0%A0%D1%83%D0%B1%D0%BB%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import pandas as pd
import re
import numpy as np

In [ ]:
pip install openpyxl==3.0.0

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving База отчета по РУБЛИКАМ.xlsx to База отчета по РУБЛИКАМ.xlsx
User uploaded file "База отчета по РУБЛИКАМ.xlsx" with length 5671633 bytes


# Текущий месяц, Обработка бонусного счета, количества подключенных клиентов и периода подключения

In [85]:
dic1= pd.read_excel('/content/drive/MyDrive/work/База отчета по РУБЛИКАМ.xlsx', sheet_name='справ')

In [108]:
cube1= pd.read_excel('/content/drive/MyDrive/work/База отчета по РУБЛИКАМ.xlsx', sheet_name='Лист2')

In [109]:
cube= cube1
cube.rename(columns={'Партнеры[Код партнера]': 'code', 'Партнеры[Статус по потенциалу]': 'status2', 'Партнеры[Подканал сбыта]': 'channel', 'Партнеры[Бюджетник]': 'gov', '[Реальные_NEW]': 'count'}, inplace=True)
cube['code'] = cube['code'].astype('int64')
cube['count'] = cube['count'].astype('int64')
cube = cube.replace({'status2':{'Крупный': '09. Крупный', 'VIP': '10. Vip&Key', 'Ключевой': '10. Vip&Key'}})
cube = cube.replace({'channel':{'КБ': '02. КБ', 'СМБ': '04. СМБ', 'ПРЦ': '03. ПРЦ'}})
cube = cube.replace({'gov':{'Да': '05. Госсектор'}})
cube.insert(1, 'срез', '01. ОКТ')
cube.head()

,code,срез,status2,channel,gov,count
0,339987,01. ОКТ,Не определен,02. КБ,Нет,1
1,309540,01. ОКТ,09. Крупный,02. КБ,Нет,1
2,107321,01. ОКТ,Не определен,02. КБ,Нет,1
3,107480,01. ОКТ,Не определен,02. КБ,Нет,1
4,336988,01. ОКТ,Не определен,02. КБ,Нет,1


In [87]:
dic= dic1
dic.rename(columns={'Код клиента': 'code', 'Подключен': 'connect', 'Бонусный счет': 'acc_sum'}, inplace=True)
dic.drop(['Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
dic['code'] = dic['code'].str.extract('(\d+)')
dic.code = pd.to_numeric(dic.code).fillna(0)
date_pattern = r'([0-9]{2}\.[0-9]{2}\.[0-9]{4})'
year_pattern = r'(\.[0-9]{4})'
month_pattern= r'(\.[0-9]{2}\.)'
dic['year'] = dic['connect'].str.extract(year_pattern)
dic['year'] = dic['year'].str.extract( r'([0-9]{4})').fillna(value=0).astype('int64')
dic['month'] =  dic['connect'].str.extract(month_pattern)
dic['month'] =  dic['month'].str.extract(r'([0-9]{2})').fillna(value=0).astype('int64')
dic['connect'] = dic['connect'].str.extract(date_pattern).astype('datetime64[ns]')
#dic['year'] = pd.DatetimeIndex(dic['connect']).year.fillna(value=0).astype('int64')
#dic['month'] = pd.DatetimeIndex(dic['connect']).month.fillna(value=0).astype('int64')
dic.head()

,code,acc_sum,connect,year,month
0,371401,763.57,2020-06-29,2020,6
1,402069,0.00,2020-06-29,2020,6
2,394377,0.00,2020-06-29,2020,6
3,398115,0.00,2020-06-29,2020,6
4,358790,1184.70,2020-06-29,2020,6


In [88]:
df = pd.merge(dic, cube, how='left')
df.head()

,code,acc_sum,connect,year,month,срез,status2,channel,gov,count
0,371401,763.57,2020-06-29,2020,6,01. ОКТ,Средний,04. СМБ,Нет,1.0
1,402069,0.00,2020-06-29,2020,6,01. ОКТ,Мелкий,04. СМБ,Нет,1.0
2,394377,0.00,2020-06-29,2020,6,01. ОКТ,Средний,04. СМБ,Нет,1.0
3,398115,0.00,2020-06-29,2020,6,01. ОКТ,Средний,04. СМБ,Нет,1.0
4,358790,1184.70,2020-06-29,2020,6,01. ОКТ,Средний,04. СМБ,Нет,1.0


In [89]:
curr_month1 = df\
              .query('year>0')\
              .groupby('срез', as_index=False)\
              .agg({'acc_sum':'sum', 'code':'count'}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [90]:
curr_month2_4 = df\
              .query('year>0 and (channel=="04. СМБ" or channel=="02. КБ" or channel=="03. ПРЦ")')\
              .groupby('channel', as_index=False)\
              .agg({'acc_sum':'sum', 'code':'count'})\
              .rename(columns={"channel":"срез"}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [91]:
curr_month5 = df\
              .query('year>0 and gov=="05. Госсектор"')\
              .groupby('gov', as_index=False)\
              .agg({'acc_sum':'sum', 'code':'count'})\
              .rename(columns={"gov":"срез"}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [92]:
curr_month910 = df\
              .query('year>0 and (status2=="09. Крупный" or status2=="10. Vip&Key")')\
              .groupby('status2', as_index=False)\
              .agg({'acc_sum':'sum', 'code':'count'})\
              .rename(columns={"status2":"срез"}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [93]:
# Установить корректный год и месяц
curr_month = pd.concat([curr_month1, curr_month2_4, curr_month5, curr_month910], axis=0, ignore_index=True)
curr_month['year']=2022
curr_month['month']=1
curr_month

,срез,acc_sum,code,year,month
0,01. ОКТ,18982083.97,92348,2022,1
1,02. КБ,8420287.36,30140,2022,1
2,03. ПРЦ,2155799.03,1996,2022,1
3,04. СМБ,8405997.58,60212,2022,1
4,05. Госсектор,1230855.82,5818,2022,1
5,09. Крупный,4156396.98,17802,2022,1
6,10. Vip&Key,4232812.62,9273,2022,1


In [94]:
programm_join1 = df\
                .query('year>0')\
                .groupby(['срез', 'year', 'month'], as_index=False)\
                .agg({'code':'count'})\
                .rename(columns={"code":"programm_join"}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов           

In [95]:
programm_join2_4 = df\
              .query('year>0 and (channel=="04. СМБ" or channel=="02. КБ" or channel=="03. ПРЦ")')\
              .groupby(['channel', 'year', 'month'], as_index=False)\
              .agg({'code':'count'})\
              .rename(columns={"channel":"срез", "code":"programm_join"}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [96]:
programm_join5 = df\
              .query('year>0 and gov=="05. Госсектор"')\
              .groupby(['gov', 'year', 'month'], as_index=False)\
              .agg({'code':'count'})\
              .rename(columns={"gov":"срез", "code":"programm_join"}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [97]:
programm_join910 = df\
              .query('year>0 and (status2=="09. Крупный" or status2=="10. Vip&Key")')\
              .groupby(['status2', 'year', 'month'], as_index=False)\
              .agg({'code':'count'})\
              .rename(columns={"status2":"срез", "code":"programm_join"}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [98]:
programm_join = pd.concat([programm_join1, programm_join2_4, programm_join5, programm_join910], axis=0, ignore_index=True)

In [132]:
programm_join.sample()

,срез,year,month,programm_join
77,04. СМБ,2022,1,233


# Изменение старого DF в новый сжатый вид

In [153]:
old_df1= pd.read_excel('/content/drive/MyDrive/work/База отчета по РУБЛИКАМ.xlsx', sheet_name='база')

In [155]:
old_df=old_df1
old_df.rename(columns={'месяц': 'date', 'Код клиента': 'code', 'Кол-во начисленных рубликов за период': 'plus_bonus', 'Кол-во списанных рубликов за период': 'minus_bonus', 'Кол-во рубликов сгорело за период': 'burn_bonus'}, inplace=True)
year_pattern = r'(\.[0-9]{4})'
month_pattern= r'(\.[0-9]{2}\.)'
old_df['year'] = old_df['date'].str.extract(year_pattern)
old_df['year'] = old_df['year'].str.extract( r'([0-9]{4})').fillna(value=0).astype('int64')
old_df['month'] =  old_df['date'].str.extract(month_pattern)
old_df['month'] =  old_df['month'].str.extract(r'([0-9]{2})').fillna(value=0).astype('int64')
old_df['code'] = old_df['code'].str.extract('(\d+)').astype('int64')
old_df.dtypes

date            object
code             int64
plus_bonus     float64
minus_bonus    float64
burn_bonus     float64
year             int64
month            int64
dtype: object

In [156]:
arch_df = pd.merge(old_df, cube, how='left')
arch_df['plus_count']=np.where(arch_df.plus_bonus != 0, 1, 0)
arch_df['minus_count']=np.where(arch_df.minus_bonus != 0, 1, 0)
arch_df['burn_count']=np.where(arch_df.burn_bonus != 0, 1, 0)
arch_df.head()

,date,code,plus_bonus,minus_bonus,burn_bonus,year,month,срез,status2,channel,gov,count,plus_count,minus_count,burn_count
0,_01.01.2021,371401,0.00,0.0,-138.49,2021,1,01. ОКТ,Средний,04. СМБ,Нет,1.0,0,0,1
1,_01.01.2021,358790,149.18,0.0,0.00,2021,1,01. ОКТ,Средний,04. СМБ,Нет,1.0,1,0,0
2,_01.01.2021,401678,0.00,0.0,-33.53,2021,1,01. ОКТ,Мелкий,04. СМБ,Нет,1.0,0,0,1
3,_01.01.2021,400635,77.41,0.0,0.00,2021,1,01. ОКТ,Средний,04. СМБ,Нет,1.0,1,0,0
4,_01.01.2021,399576,31.42,0.0,0.00,2021,1,01. ОКТ,Мелкий,04. СМБ,Нет,1.0,1,0,0


In [157]:
arch_df1 = arch_df\
              .query('year>0')\
              .groupby(['срез', 'year', 'month'], as_index=False)\
              .agg({'plus_bonus':'sum', 'minus_bonus':'sum', 'burn_bonus':'sum', 'plus_count':'sum', 'minus_count':'sum', 'burn_count':'sum'}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [158]:
arch_df24 = arch_df\
              .query('year>0 and (channel=="04. СМБ" or channel=="02. КБ" or channel=="03. ПРЦ")')\
              .groupby(['channel', 'year', 'month'], as_index=False)\
              .agg({'plus_bonus':'sum', 'minus_bonus':'sum', 'burn_bonus':'sum', 'plus_count':'sum', 'minus_count':'sum', 'burn_count':'sum'})\
              .rename(columns={"channel":"срез"}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [159]:
arch_df5 = arch_df\
              .query('year>0 and gov=="05. Госсектор"')\
              .groupby(['gov', 'year', 'month'], as_index=False)\
              .agg({'plus_bonus':'sum', 'minus_bonus':'sum', 'burn_bonus':'sum', 'plus_count':'sum', 'minus_count':'sum', 'burn_count':'sum'})\
              .rename(columns={"gov":"срез"}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [160]:
arch_df910 = arch_df\
               .query('year>0 and (status2=="09. Крупный" or status2=="10. Vip&Key")')\
              .groupby(['status2', 'year', 'month'], as_index=False)\
              .agg({'plus_bonus':'sum', 'minus_bonus':'sum', 'burn_bonus':'sum', 'plus_count':'sum', 'minus_count':'sum', 'burn_count':'sum'})\
              .rename(columns={"status2":"срез"}) #не учитываются бонусы на счетах у отключенных от ПЛ клиентов #не учитываются бонусы на счетах у отключенных от ПЛ клиентов

In [161]:
arch_total = pd.concat([arch_df1, arch_df24, arch_df5, arch_df910], axis=0, ignore_index=True)
arch_total

,срез,year,month,plus_bonus,minus_bonus,burn_bonus,plus_count,minus_count,burn_count
0,01. ОКТ,2020,6,1305.04,0.00,0.00,11,0,0
1,01. ОКТ,2020,7,26674.45,0.00,0.00,87,0,0
2,01. ОКТ,2020,8,67250.55,-10725.91,0.00,266,4,0
3,01. ОКТ,2020,9,334841.37,-11442.72,0.00,1743,7,0
4,01. ОКТ,2020,10,980734.20,-42875.36,0.00,5246,15,0
...,...,...,...,...,...,...,...,...,...
134,10. Vip&Key,2021,9,631283.93,-381679.42,-22636.08,1452,137,67
135,10. Vip&Key,2021,10,619868.76,-305060.77,-56153.56,1419,105,106
136,10. Vip&Key,2021,11,591203.14,-330939.70,-23740.11,1449,128,73
137,10. Vip&Key,2021,12,747723.95,-486185.83,-38795.87,1587,155,79


In [162]:
arch_total.to_excel('/content/drive/MyDrive/work/Archive_dataframe_bonus.xlsx', index=False)

# Объединение архивного ДФ,периода подключения и текущего месяца

In [163]:
arch_total1= pd.read_excel('/content/drive/MyDrive/work/Archive_dataframe_bonus.xlsx')

In [164]:
full_base = pd.merge(arch_total1, programm_join, how='outer')

In [165]:
full_base.head()

,срез,year,month,plus_bonus,minus_bonus,burn_bonus,plus_count,minus_count,burn_count,programm_join
0,01. ОКТ,2020,6,1305.04,0.00,0.0,11.0,0.0,0.0,195.0
1,01. ОКТ,2020,7,26674.45,0.00,0.0,87.0,0.0,0.0,7.0
2,01. ОКТ,2020,8,67250.55,-10725.91,0.0,266.0,4.0,0.0,887.0
3,01. ОКТ,2020,9,334841.37,-11442.72,0.0,1743.0,7.0,0.0,56588.0
4,01. ОКТ,2020,10,980734.20,-42875.36,0.0,5246.0,15.0,0.0,5456.0


In [167]:
full_base = pd.merge(full_base, curr_month, how='outer')

In [168]:
full_base.to_excel('/content/drive/MyDrive/work/full_base.xlsx', index=False)